In [24]:
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools

In [112]:
class ELM:
    """
    Extreme Learning Machine for a single-hidden-layer feedforward neural network (SLFN).
    Steps (Huang et al., 2006):
      1) Randomly assign input weights W1 and biases b for the hidden layer.
      2) Compute hidden layer output matrix H.
      3) Compute output weights beta = H^dagger * T (Moore-Penrose pseudoinverse).
    """
    def __init__(self, input_size, hidden_size, output_size, activation='relu', seed=None):
        """
        Parameters
        ----------
        input_size : int
            Number of input features (dimension of x).
        hidden_size : int
            Number of hidden neurons.
        output_size : int
            Number of output dimensions (dimension of t).
        activation : callable, optional
            Activation function g(z) to use.
            If None, defaults to ReLU.
        seed : int, optional
            Seed for reproducible random initialization.
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        if seed:
            np.random.seed(seed)

        # Choose activation
        if isinstance(activation, str):
            if activation.lower() == 'relu':
                self.activation = self._relu
                self.activation_deriv = self._relu_derivative
            elif activation.lower() == 'tanh':
                self.activation = self._tanh
                self.activation_deriv = self._tanh_derivative
            else:
                raise ValueError("Unsupported activation string. Use 'relu' or 'tanh'.")
        else:
            # user-supplied function
            self.activation = activation
            # no derivative provided => handle carefully or raise error
            self.activation_deriv = None
            print("Warning: no derivative for a custom activation. Backprop may fail.")

        # Randomly init input->hidden weights, not updated in ELM
        # For ReLU, a good approach is He initialization:
        # For Tanh, a good approach is Xavier (scaled uniform).
        if isinstance(activation, str) and activation.lower() == 'relu':
            # He initialization
            he_std = np.sqrt(2.0 / input_size)
            self.weights_input_hidden = np.random.normal(
                loc=0.0, scale=he_std, size=(input_size, hidden_size)
            )
            self.bias_hidden = np.random.normal(
                loc=0.0, scale=1e-2, size=(1, hidden_size)
            )
        else:
            # e.g. Tanh => Xavier
            limit = np.sqrt(6.0 / (input_size + hidden_size))
            self.weights_input_hidden = np.random.uniform(
                low=-limit, high=limit, size=(input_size, hidden_size)
            )
            self.bias_hidden = np.random.uniform(
                low=-limit, high=limit, size=(1, hidden_size)
            )

        # Hidden->output weights: We DO train these
        # We'll do a simple Xavier-like approach for either ReLU or Tanh:
        limit_out = np.sqrt(6.0 / (hidden_size + output_size))
        self.weights_hidden_output = np.random.uniform(
            low=-limit_out, high=limit_out, size=(hidden_size, output_size)
        )
        self.bias_output = np.random.uniform(
            low=-limit_out, high=limit_out, size=(1, output_size)
        )

        # Placeholders for forward pass
        self.hidden_layer_input = None
        self.hidden_layer_output = None
        self.output_layer_input = None
        self.predicted_output = None

    # ------------------------
    # Activation functions
    # ------------------------
    def _relu(self, x):
        return np.maximum(0, x)

    def _relu_derivative(self, x):
        # derivative wrt the pre-activation
        return (x > 0).astype(float)

    def _tanh(self, x):
        return np.tanh(x)

    def _tanh_derivative(self, x):
        # derivative wrt pre-activation for tanh
        # if we define y = tanh(x), derivative = 1 - y^2
        y = np.tanh(x)
        return 1.0 - y*y

    # L1 subgradient
    def _l1_subgrad(self, w):
        # returns sign(w), with sign(0)=0
        grad = np.zeros_like(w)
        grad[w > 0] = 1.0
        grad[w < 0] = -1.0
        return grad

    # ------------------------
    # Forward pass
    # ------------------------
    def forward(self, X):
        """
        Forward pass with either ReLU or tanh hidden activation,
        then a linear activation (or if you prefer, you could
        also do tanh at the output).
        """
        # hidden
        self.hidden_layer_input = X.dot(self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = self.activation(self.hidden_layer_input)

        # output
        self.output_layer_input = self.hidden_layer_output.dot(self.weights_hidden_output) + self.bias_output
        # We'll do linear output by default.
        # If you want tanh final, do: self.predicted_output = np.tanh(self.output_layer_input)
        self.predicted_output = self.output_layer_input

        return self.predicted_output

    # ------------------------
    # Backward pass
    # ------------------------
    def backward(self, X, y):
        """
        Compute gradients wrt the hidden->output weights
        for a MSE + L1 penalty on W2.

        Because we do not update input->hidden in an ELM
        (by definition it's random and fixed),
        we only compute partial derivatives wrt (W2, b2).
        """
        n_samples = X.shape[0]

        # 1) dLoss/d(output)
        # MSE derivative: (pred - y)
        output_error = (self.predicted_output - y)  # shape (n_samples, output_size)

        # 2) derivative wrt W2, b2
        # hidden_layer_output shape: (n_samples, hidden_size)
        dW2 = (self.hidden_layer_output.T @ output_error) / n_samples
        db2 = np.sum(output_error, axis=0, keepdims=True) / n_samples

        # L1 subgradient on W2
        if self.l1_lambda > 1e-15:
            dW2 += self.l1_lambda * self._l1_subgrad(self.weights_hidden_output)

        return dW2, db2

    # ------------------------
    # Update weights
    # ------------------------
    def update(self, dW2, db2, lr=1e-3):
        """
        Gradient descent step on hidden->output layer
        """
        self.weights_hidden_output -= lr * dW2
        self.bias_output -= lr * db2

    # ------------------------
    # Evaluate
    # ------------------------
    def evaluate_loss(self, X, y):
        """
        Return MSE + L1 penalty for the forward pass.
        MSE = 0.5 * mean( (pred - y)^2 )
        plus L1 = lambda * sum(|W2|)
        ignoring W1 since not trained.
        """
        pred = self.forward(X)
        mse = 0.5 * np.mean((pred - y)**2)
        l1_term = self.l1_lambda * np.sum(np.abs(self.weights_hidden_output))
        return mse + l1_term

    def predict(self, X):
        """
        Just forward pass, ignoring state variables
        """
        hidden = self.activation(X.dot(self.weights_input_hidden) + self.bias_hidden)
        # linear output
        output = hidden.dot(self.weights_hidden_output) + self.bias_output
        return output

In [114]:
def generate_synthetic_data(d=100, N=1000, hidden_dim=50, sigma=0.1, seed=None):
    """
    Generates a synthetic dataset following the described process.

    Parameters:
    d (int): Number of input features.
    N (int): Number of samples.
    hidden_dim (int): Number of hidden neurons.
    sigma (float): Standard deviation of noise.
    seed (int, optional): Random seed for reproducibility.

    Returns:
    X (ndarray): Input feature matrix of shape (d, N).
    y (ndarray): Target values of shape (N,).
    """
    if seed is not None:
        np.random.seed(seed)

    # Generate input features X, half from normal and half from uniform distribution
    X = np.zeros((N, d))
    half_d = d // 2
    X[:, :half_d] = np.random.normal(0, 1, (N, half_d))  # Normal distribution
    X[:, half_d:] = np.random.uniform(-1, 1, (N, d - half_d))  # Uniform distribution


    # Generate random weight matrices and bias vector
    W1 = np.random.randn(hidden_dim, d)
    b1 = np.random.randn(1, hidden_dim) 
    W2_star = np.random.randn(hidden_dim, 1) 


    # Compute hidden layer activation
    hidden_activation = np.tanh(X @ W1.T + b1)  # Now correct: (N, hidden_dim)

    # Compute target values with noise
    noise = np.random.normal(0, sigma, (N,1))
    y = hidden_activation @ W2_star + noise  # Now correctly (N, 1)

    return X, y

In [183]:
def train_smoothed_gradient(
    elm,
    X_train,
    y_train,
    epochs,
    base_lr,
    momentum_coef, #Ok
    lambda_reg=0.01, # We can test more values
    mu_init=1e-2, # it should be epsilon/mu but it is the rule of thumbs for defining\mu
    mu_decay=0.5,
    L_mse_estimate=1.0,
    early_stop_patience=20,
    early_stop_tol=1e-4,
    gradient_norm_threshold=None
):
    """
    Train the ELM model using a smoothed L1 gradient approach with dynamic mu and LR.

    Instead of the non-differentiable L1 norm,
    we replace |W2| by sqrt(W2^2 + mu^2),
    whose subgradient wrt W2 is:
         W2 / sqrt(W2^2 + mu^2).

    We also adjust the smoothing parameter mu and the learning rate lr
    each epoch to stay aligned with typical smoothing and momentum guidelines.

    Parameters
    ----------
    elm : ELM
        Must provide:
            - weights_hidden_output (numpy array)
            - bias_output (numpy array)
            - forward(X)
            - backward(X, y) -> (dW2, dB2)
            - evaluate_loss(X, y)
    X_train, y_train : np.ndarray
        Training data, shape is consistent with the ELM design.
    epochs : int
        Number of epochs to train.
    base_lr : float
        Base (initial) learning rate factor.
    momentum_coef : float
        Momentum coefficient in [0, 1).
    lambda_reg : float
        L1 regularization strength for W2.
    mu_init : float
        Initial smoothing parameter, typically ~1e-2 or 1e-3.
    mu_decay_gamma : float
        Exponent for the decay schedule, e.g. 0.5 for inverse-sqrt, 1.0 for inverse-linear.
    L_mse_estimate : float
        Estimated Lipschitz constant for the MSE gradient.
    early_stop_patience : int
        Number of epochs with no significant improvement before early stop.
    early_stop_tol : float
        Relative improvement threshold for early stop check.
    gradient_norm_threshold : float or None
        If not None, stop when the total gradient norm < threshold.

    Returns
    -------
    train_loss_history : list of floats
        Training loss per epoch.
    """
    # Momentum buffers
    v_weights = np.zeros_like(elm.weights_hidden_output)
    v_bias = np.zeros_like(elm.bias_output)

    train_loss_history = []

    for epoch in range(epochs):
        # Update smoothing parameter mu (e.g. inverse-sqrt decay)
        mu_k = mu_init / ((epoch + 1) ** mu_decay)

        # The effective Lipschitz for the smoothed L1 portion ~ lambda_reg / mu_k
        L_smooth_l1 = lambda_reg / max(mu_k, 1e-12)

        # Combine with the MSE Lipschitz estimate
        L_eff = L_mse_estimate + L_smooth_l1

        # Here, we use a simple fixed base_lr to keep code concise.
        lr_k = base_lr

        # Forward pass
        elm.forward(X_train)
        train_loss = elm.evaluate_loss(X_train, y_train)
        train_loss_history.append(train_loss)

        # Compute MSE gradient
        dW2_mse, db2_mse = elm.backward(X_train, y_train)

        # Smoothed L1 gradient for W2
        W2_smooth = elm.weights_hidden_output
        denom = np.sqrt(W2_smooth * W2_smooth + mu_k * mu_k)
        denom = np.where(denom < 1e-12, 1e-12, denom)  # avoid dividing by zero
        dW2_smooth_l1 = lambda_reg * (W2_smooth / denom)

        # Combine gradients
        dW2_total = dW2_mse + dW2_smooth_l1
        db2_total = db2_mse  # no L1 penalty on bias

        # Optional gradient norm stopping criterion
        if gradient_norm_threshold is not None:
            grad_norm = np.linalg.norm(dW2_total) + np.linalg.norm(db2_total)
            if grad_norm < gradient_norm_threshold:
                print(f"[Early Stop] Gradient norm ({grad_norm:.2e}) below threshold.")
                break

        # Heavy-ball momentum update
        v_weights = momentum_coef * v_weights - lr_k * dW2_total
        v_bias    = momentum_coef * v_bias    - lr_k * db2_total

        # Apply updates
        elm.weights_hidden_output += v_weights
        elm.bias_output           += v_bias

    return train_loss_history

In [185]:
class TestOptimizerELM:
    def __init__(self, elm_model, optimizer, loss_function=None, grad_function=None):
      """
      Generalized optimizer testing framework.

      Parameters:
      - elm_model: An instance of an ELM model.
      - optimizer: A function that performs optimization (e.g., Momentum, Adam).
      - loss_function: The loss function to optimize (default: Ackley function).
      - grad_function: The gradient of the loss function (default: Ackley gradient).
      """
      self.elm = elm_model
      self.optimizer = optimizer
      self.loss_function = loss_function or self.ackley
      self.grad_function = grad_function or self.ackley_grad

    @staticmethod
    def ackley(x):
      """Computes the Ackley function."""
      a, b, c = 20, 0.2, 2 * np.pi
      n = len(x)
      sum1 = np.sum(x**2)
      sum2 = np.sum(np.cos(c * x))
      term1 = -a * np.exp(-b * np.sqrt(sum1 / n))
      term2 = -np.exp(sum2 / n)
      return term1 + term2 + a + np.e

    @staticmethod
    def ackley_grad(x):
      """Computes the gradient of the Ackley function."""
      a, b, c = 20, 0.2, 2 * np.pi
      n = len(x)
      term1 = (b / np.sqrt(n * np.sum(x**2))) * x * a * np.exp(-b * np.sqrt(np.sum(x**2) / n))
      term2 = (c / n) * np.sin(c * x) * np.exp(np.sum(np.cos(c * x)) / n)
      return term1 + term2

    def evaluate_loss(self):
      """Evaluates the loss function on the model's weights."""
      return self.loss_function(self.elm.weights_hidden_output.flatten())

    def compute_gradient(self):
      """Computes gradients for optimization."""
      return self.grad_function(self.elm.weights_hidden_output.flatten()).reshape(self.elm.weights_hidden_output.shape)

    def test_optimizer(self, X_train, y_train, epochs=500, lr=0.01, momentum_coef=0.9):
      """
      Tests the optimizer by minimizing the loss function.

      Parameters:
      - X_train, y_train: Training dataset (can be dummy).
      - epochs: Number of training iterations.
      - lr: Learning rate.
      - momentum_coef: Momentum coefficient (if applicable).
      """
      train_loss_history = self.optimizer(self.elm, X_train, y_train, epochs, lr, momentum_coef)

      # Visualization
      import matplotlib.pyplot as plt
      plt.figure(figsize=(10, 6))
      plt.plot(train_loss_history, label="Optimizer Performance on Ackley")
      plt.xlabel("Epoch")
      plt.ylabel("Loss")
      plt.title("Convergence of Optimizer on Ackley Function")
      plt.legend()
      plt.grid()
      plt.show()

      final_loss = train_loss_history[-1]
      final_weights = self.elm.weights_hidden_output.flatten()

      print(f"\nFinal loss: {final_loss:.6f} (Expected: ~0)")
      print(f"Final weight values: {final_weights} (Expected: Close to [0,0])")

      # Final correctness check
      assert final_loss < 1e-2, "Test failed: Final loss is too high!"
      assert np.linalg.norm(final_weights) < 1e-2, "Test failed: Final weights are not near zero!"

      print("✅ Test Passed: Optimizer correctly minimizes the given function!")


# Example usage
class ELM_Ackley(ELM):
    def __init__(self, input_size, hidden_size, output_size, seed=None):
      super().__init__(input_size, hidden_size, output_size, activation='relu', seed=seed)

    def evaluate_loss(self, X, y=None):
      return TestOptimizerELM.ackley(self.weights_hidden_output.flatten())

    def backward(self, X, y=None):
      grad = TestOptimizerELM.ackley_grad(self.weights_hidden_output.flatten()).reshape(self.weights_hidden_output.shape)
      return grad, np.zeros_like(self.bias_output)

class ELM_Quadratic(ELM):


    def __init__(self, input_size, hidden_size, output_size, seed=None):
        # We reuse the base ELM's constructor for random initialization,
        # but we won't actually use the forward pass for anything. 
        super().__init__(input_size, hidden_size, output_size, activation='relu', seed=seed)

    def evaluate_loss(self, X, y=None):

        # shape of weights_hidden_output = (hidden_size, output_size)
        W2 = self.weights_hidden_output
        return 0.5 * np.sum(W2 * W2)

    def backward(self, X, y=None):

        grad_w2 = self.weights_hidden_output  # shape = (hidden_size, output_size)
        grad_b2 = np.zeros_like(self.bias_output)  # no contribution in the loss
        return grad_w2, grad_b2

In [187]:
# ------------------------
# Generic Optimizer Tester
# ------------------------
class OptimizerTester:
    def __init__(self, optimizer_func):
        """
        Parameters:
          optimizer_func : A function that trains an ELM instance and returns
                           the training loss history. It must have the same
                           interface as the current train_momentum function.
        """
        self.optimizer = optimizer_func

    # -------------------------------
    # 1. Unit test: Update Equations
    # -------------------------------
    def test_update_equations_on_quadratic(self):
        print("Running update equation test on quadratic function:")
        input_size, hidden_size, output_size = 10, 5, 1
        elm = ELM_Quadratic(input_size, hidden_size, output_size, seed=123)
        # For a deterministic check, set weights to ones.
        initial_weights = np.ones((hidden_size, output_size))
        elm.weights_hidden_output = initial_weights.copy()

        # Use one update step with lr=0.01 and momentum_coef=0.9.
        lr = 0.01
        momentum_coef = 0.9
        epochs = 1
        X_dummy = np.zeros((10, input_size))
        y_dummy = np.zeros((10, output_size))

        _ = self.optimizer(elm, X_dummy, y_dummy, epochs, lr, momentum_coef)

        # Expected update:
        # For the quadratic loss f(W)=0.5*||W||^2, the gradient is W.
        # With one iteration: v = -lr * grad = -0.01 * 1 = -0.01,
        # so the new weight should be 1 - 0.01 = 0.99.
        expected_weights = 0.99 * initial_weights
        actual_weights = elm.weights_hidden_output
        error = np.linalg.norm(actual_weights - expected_weights)

        print(f"Expected weights:\n{expected_weights}")
        print(f"Actual weights:\n{actual_weights}")
        print(f"Update error: {error:.6f}\n")
        assert error < 1e-1, "Update equations test failed!"

    # --------------------------------------------------
    # 2. Convergence Test on a Quadratic Function
    # --------------------------------------------------
    def test_convergence_quadratic(self):
        print("Running convergence test on quadratic function:")
        input_size, hidden_size, output_size = 10, 5, 1
        elm = ELM_Quadratic(input_size, hidden_size, output_size, seed=123)
        # Start with random weights.
        elm.weights_hidden_output = np.random.randn(hidden_size, output_size)

        X_dummy = np.zeros((10, input_size))
        y_dummy = np.zeros((10, output_size))

        epochs = 200
        lr = 0.01
        momentum_coef = 0.9
        losses = self.optimizer(elm, X_dummy, y_dummy, epochs, lr, momentum_coef)

        print(f"Initial loss: {losses[0]:.6f}")
        print(f"Final loss: {losses[-1]:.6f}\n")
        # Since the minimum is zero, the final loss should be very small.
        assert losses[-1] < 1e-6, "Quadratic convergence test failed!"

    # --------------------------------------------------
    # 3. Convergence Test on the Ackley Function
    # --------------------------------------------------
    def test_convergence_ackley(self):
        print("Running convergence test on Ackley function:")
        input_size, hidden_size, output_size = 10, 20, 1
        elm = ELM_Ackley(input_size, hidden_size, output_size, seed=123)

        # For the Ackley function, the test is that the loss decreases relative to the start.
        X_dummy = np.zeros((10, input_size))
        y_dummy = np.zeros((10, output_size))

        epochs = 200
        lr = 0.01
        momentum_coef = 0.9
        losses = self.optimizer(elm, X_dummy, y_dummy, epochs, lr, momentum_coef)

        print(f"Initial Ackley loss: {losses[0]:.6f}")
        print(f"Final Ackley loss: {losses[-1]:.6f}\n")
        assert losses[-1] < losses[0], "Ackley convergence test failed: Loss did not decrease!"

    # --------------------------------------------------
    # 4. Test: Gradient Norm Decrease on Quadratic Function
    # --------------------------------------------------
    def test_gradient_norm_decrease_quadratic(self):
        print("Running gradient norm decrease test on quadratic function:")
        input_size, hidden_size, output_size = 10, 5, 1
        elm = ELM_Quadratic(input_size, hidden_size, output_size, seed=123)
        elm.weights_hidden_output = np.random.randn(hidden_size, output_size)

        X_dummy = np.zeros((10, input_size))
        y_dummy = np.zeros((10, output_size))

        # Compute the initial gradient norm.
        initial_grad, _ = elm.backward(X_dummy, y_dummy)
        initial_grad_norm = np.linalg.norm(initial_grad)

        # Run the optimizer for a number of epochs.
        epochs = 200
        lr = 0.01
        momentum_coef = 0.9
        _ = self.optimizer(elm, X_dummy, y_dummy, epochs, lr, momentum_coef)

        # Compute the final gradient norm.
        final_grad, _ = elm.backward(X_dummy, y_dummy)
        final_grad_norm = np.linalg.norm(final_grad)

        print(f"Initial gradient norm: {initial_grad_norm:.6f}")
        print(f"Final gradient norm: {final_grad_norm:.6f}\n")
        assert final_grad_norm < initial_grad_norm, "Gradient norm did not decrease!"

    # -------------------------------
    # Run all tests
    # -------------------------------
    def run_all_tests(self):
        self.test_update_equations_on_quadratic()
        self.test_convergence_quadratic()
        self.test_convergence_ackley()
        self.test_gradient_norm_decrease_quadratic()
        print("All tests passed successfully!")

# Example usage:
# Assume that train_momentum is defined as in your current implementation.
# To test the momentum-based optimizer:
tester = OptimizerTester(train_smoothed_gradient)
tester.run_all_tests()

Running update equation test on quadratic function:
Expected weights:
[[0.99]
 [0.99]
 [0.99]
 [0.99]
 [0.99]]
Actual weights:
[[0.9899]
 [0.9899]
 [0.9899]
 [0.9899]
 [0.9899]]
Update error: 0.000224

Running convergence test on quadratic function:
Initial loss: 2.115035
Final loss: 0.000000

Running convergence test on Ackley function:
Initial Ackley loss: 3.048249
Final Ackley loss: 0.004254

Running gradient norm decrease test on quadratic function:
Initial gradient norm: 2.056713
Final gradient norm: 0.000024

All tests passed successfully!


## Experiments

In [ ]:
# =============================================================================
# Experiment Evaluation Function for Momentum Approach
# =============================================================================
def evaluate_experiments_momentum(X_train, y_train, X_val, y_val,
                                  input_size, hidden_size, output_size,
                                  epochs, seed):
    """
    Run experiments over a grid of hyperparameters for the momentum approach.

    We vary:
      - Learning rate (lr)
      - Momentum coefficient (eta)
      - Regularization parameter lambda (l1_lambda)
      - Gradient noise levels

    Metrics collected include:
      - Convergence speed (loss curves)
      - Final loss values (train and validation)
      - Sparsity of W2 (fraction of weights with abs(value) < threshold)
      - Computational efficiency (total training time and avg epoch time)
      - Robustness (performance variation under different noise levels)
    """
    learning_rates = [1e-5, 0.001, 0.01, 0.05, 0.1]
    momentum_coefs = [0.8, 0.9, 0.99]
    lambda_values = [1e-4, 1e-3, 1e-2, 1e-1]  # test different L1 regularization strengths

    results = {}

    for lam, lr, momentum in itertools.product(lambda_values, learning_rates, momentum_coefs):
        # Initialize the model (same architecture, fixed seed for reproducibility)
        elm_model = ELM(input_size, hidden_size, output_size, activation='relu', seed=seed)
        elm_model.l1_lambda = lam

        start_time = time.time()
        train_losses = train_smoothed_gradient(elm_model, X_train, y_train, epochs, lr, momentum, lambda_reg=lam)
            
        val_losses =  train_smoothed_gradient(elm_model, X_train, y_train, epochs, lr, momentum, lambda_reg=lam)
        
        total_time = time.time() - start_time
        avg_epoch_time = total_time / epochs

        # Compute sparsity of W2: fraction of weights with abs(value) below a small threshold.
        threshold = 1e-3
        sparsity = np.mean(np.abs(elm_model.weights_hidden_output) < threshold)

        config = ('momentum', lam, lr, momentum)
        results[config] = {
            "train_losses": train_losses,
            "val_losses": val_losses,
            "total_time": total_time,
            "avg_epoch_time": avg_epoch_time,
            "final_sparsity": sparsity,
            "final_train_loss": train_losses[-1],
            "final_val_loss": val_losses[-1]
        }
        print(f"[Momentum] λ={lam}, lr={lr}, momentum={momentum} => "
              f"Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, "
              f"Sparsity: {sparsity*100:.2f}%, Total Time: {total_time:.2f}s")

    return results

# --- Synthetic Data Experiment ---
input_size = 100
n_samples = 5000
hidden_size = 50
output_size = 1
epochs = 500
sigma = 0.1
seed = 42
X_syn, y_syn = generate_synthetic_data(
    d=input_size, # n input features
    N=n_samples, # number of samples
    hidden_dim=hidden_size, # n hidden layers
    sigma=sigma, # std dev of noise
    seed=seed)

split_index = int(0.8 * n_samples)
X_train_syn, y_train_syn = X_syn[:split_index], y_syn[:split_index]
X_val_syn, y_val_syn = X_syn[split_index:], y_syn[split_index:]

results_synthetic = evaluate_experiments_momentum(
    X_train_syn,
    y_train_syn,
    X_val_syn,
    y_val_syn,
    input_size,
    hidden_size,
    output_size,
    epochs,
    seed)

# Plot convergence curves for a subset of experiments (e.g. no gradient noise) for clarity.
plt.figure(figsize=(12, 8))

for config, metrics in results_synthetic.items():
    algo, lam, lr_val, momentum = config
    plt.plot(metrics["train_losses"], label=f"λ={lam}, lr={lr_val}, momentum={momentum}")

plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Momentum Convergence Curves on Synthetic Data")
plt.legend()
plt.grid(True)
plt.show()

# Summary table of key metrics.
print("\nSummary of Experimental Results (Synthetic Data):")
header = ("Algorithm", "λ", "lr", "Momentum", "Final Train Loss",
          "Final Val Loss", "Sparsity (%)", "Total Time (s)", "Avg Epoch Time (s)")
print("\t".join(str(h) for h in header))
for config, metrics in results_synthetic.items():
    algo, lam, lr_val, momentum = config
    print(f"{algo}\t{lam:.1e}\t{lr_val:.3f}\t{momentum:.2f}\t\t"
          f"{metrics['final_train_loss']:.4f}\t\t{metrics['final_val_loss']:.4f}\t\t"
          f"{metrics['final_sparsity']*100:.2f}\t\t{metrics['total_time']:.2f}\t\t"
          f"{metrics['avg_epoch_time']:.4f}")

# Print information at the end of the descent
# - final value
# - difference
# - convergence speed
print()

[Momentum] λ=0.0001, lr=1e-05, momentum=0.8 => Train Loss: 18.8217, Val Loss: 18.7072, Sparsity: 0.00%, Total Time: 4.74s
[Momentum] λ=0.0001, lr=1e-05, momentum=0.9 => Train Loss: 18.7096, Val Loss: 18.4916, Sparsity: 0.00%, Total Time: 4.69s
[Momentum] λ=0.0001, lr=1e-05, momentum=0.99 => Train Loss: 17.3265, Val Loss: 16.2613, Sparsity: 0.00%, Total Time: 4.80s
[Momentum] λ=0.0001, lr=0.001, momentum=0.8 => Train Loss: 14.3655, Val Loss: 13.6783, Sparsity: 0.00%, Total Time: 4.71s
[Momentum] λ=0.0001, lr=0.001, momentum=0.9 => Train Loss: 13.6726, Val Loss: 13.4740, Sparsity: 0.00%, Total Time: 4.67s
[Momentum] λ=0.0001, lr=0.001, momentum=0.99 => Train Loss: 13.4701, Val Loss: 13.4479, Sparsity: 0.00%, Total Time: 4.81s
[Momentum] λ=0.0001, lr=0.01, momentum=0.8 => Train Loss: 13.4505, Val Loss: 13.4482, Sparsity: 0.00%, Total Time: 4.79s
[Momentum] λ=0.0001, lr=0.01, momentum=0.9 => Train Loss: 13.4482, Val Loss: 13.4477, Sparsity: 0.00%, Total Time: 4.67s
[Momentum] λ=0.0001, lr=